In [67]:
import pandas as pd
import numpy as np
import folium
pd.set_option('display.max_columns',100)

In [68]:
profile_2018 = pd.read_csv('Chicago_Public_Schools_-_School_Profile_Information_SY1718.csv')

In [69]:
header = ['School_ID', 'School_Name', 'Network', 'Total', 'White_Student_Count', 
          'White_Student_Pct', 'African_American_Count', 'African_America_Pct', 
         'AAPI_Count', 'AAPI_Pct', 'Native_American_Count', 'Native_American_Pct',
          'Hispanic_Student_Count', 'Hispanic_Student_Pct', 
          'Multi-Racial_Student_Count', 'Multi-Racial_Student_Pct', 
          'Asian_Student_Count', 'Asian_Student_Pct', 'Hawaiian_PI_Count', 
          'Hawaiian_PI_Pct', 'Not_Available_Count', 'Not_Available_Pct']


In [70]:
demo_data_2018 = pd.read_excel('Demographics_RacialEthnic_2018.xls', 'Schools', skiprows=2, names = header)
demo_data_2018['School_ID'].astype(int, inplace=True)

0      610212
1      610209
2      609836
3      609729
4      610083
5      610564
6      610515
7      609901
8      610523
9      610137
10     609874
11     610163
12     609737
13     609857
14     610182
15     609972
16     609798
17     609766
18     609912
19     609796
20     610089
21     610111
22     609937
23     610159
24     610127
25     609792
26     609810
27     610354
28     609749
29     609744
        ...  
630    400176
631    610572
632    610573
633    400164
634    400137
635    610569
636    610570
637    610571
638    610557
639    610568
640    400173
641    400123
642    400133
643    400124
644    400125
645    400126
646    400127
647    400128
648    400150
649    400130
650    400131
651    400134
652    400135
653    400136
654    400129
655    400139
656    400141
657    400143
658    400144
659    400145
Name: School_ID, Length: 660, dtype: int64

In [71]:
profile_merge = pd.merge(demo_data_2018[['Network', 'School_ID']], profile_2018, how='inner', on='School_ID')

In [72]:
grouped_1 = profile_merge.groupby('Network_x')
network_totals = grouped_1['Student_Count_Total'].sum()
black_student_totals = grouped_1['Student_Count_Black'].sum()
hispanic_student_totals = grouped_1['Student_Count_Hispanic'].sum()
white_student_totals = grouped_1['Student_Count_White'].sum()
asian_student_totals = grouped_1['Student_Count_Asian'].sum()
native_american_totals = grouped_1['Student_Count_Native_American'].sum()
hawaiian_pi_totals = grouped_1['Student_Count_Hawaiian_Pacific_Islander'].sum()
asian_pi_totals = grouped_1['Student_Count_Asian_Pacific_Islander'].sum()
other_ethnicity_totals = grouped_1['Student_Count_Other_Ethnicity'].sum()
multi_totals = grouped_1['Student_Count_Multi'].sum()
na_totals = grouped_1['Student_Count_Ethnicity_Not_Available'].sum()

df = {'Network_Totals': network_totals, 'Black_Student_Totals': black_student_totals,
     'Hispanic_Student_Totals': hispanic_student_totals, 'White_Student_Totals': 
      white_student_totals, 'Asian_Student_Totals': asian_student_totals, 
     'Native_American_Student_Totals': native_american_totals, 'Hawaiian_Pacific_Islander_Totals':
     hawaiian_pi_totals, 'Asian_Pacific_Islander_Totals': asian_pi_totals, 'Other_Ethnicity_Totals':
     other_ethnicity_totals, 'Multi_Ethnicity_Totals': multi_totals, 
      'Ethnicity_NA_Totals': na_totals}

df = pd.DataFrame(df)

In [73]:
df.drop('Other_Ethnicity_Totals', axis=1, inplace=True)
df['Totals-Ethnicity_NA'] = df['Network_Totals']-df['Ethnicity_NA_Totals']
df.drop('Ethnicity_NA_Totals', axis=1, inplace=True)

In [74]:
row_total = []
test = df['Totals-Ethnicity_NA']
for n in range(len(df.index)):
    network_total = df.iloc[n, 9]
    x = df.iloc[n, 1:9]
    row_total.append(sum([(j/network_total)*np.log(1/(j/network_total)) for j in x if j!=0]))
df['Network_Entropy'] = np.array(row_total)

In [75]:
profile_merge['Network_Entropy'] = profile_merge['Network_x']
network_1 = df.loc['Network 1', 'Network_Entropy']
network_2 = df.loc['Network 2', 'Network_Entropy']
network_3 = df.loc['Network 3', 'Network_Entropy']
network_4 = df.loc['Network 4', 'Network_Entropy']
network_5 = df.loc['Network 5', 'Network_Entropy']
network_6 = df.loc['Network 6', 'Network_Entropy']
network_7 = df.loc['Network 7', 'Network_Entropy']
network_8 = df.loc['Network 8', 'Network_Entropy']
network_9 = df.loc['Network 9', 'Network_Entropy']
network_10 = df.loc['Network 10', 'Network_Entropy']
network_11 = df.loc['Network 11', 'Network_Entropy']
network_12 = df.loc['Network 12', 'Network_Entropy']
network_13 = df.loc['Network 13', 'Network_Entropy']
ausl = df.loc['AUSL', 'Network_Entropy']
charter = df.loc['Charter', 'Network_Entropy']
contract = df.loc['Contract', 'Network_Entropy']
isp = df.loc['ISP', 'Network_Entropy']
options = df.loc['Options', 'Network_Entropy']

mapdict = {'Network 1': network_1, 'Network 2': network_2, 'Network 3': network_3,
          'Network 4': network_4, 'Network 5': network_5, 'Network 6': network_6,
          'Network 7': network_7, 'Network 8': network_8, 'Network 9': network_9, 
          'Network 10': network_10, 'Network 11': network_11, 'Network 12': network_12,
          'Network 13': network_13, 'AUSL': ausl, 'Charter': charter, 'Contract': contract, 
          'ISP': isp, 'Options':options}
profile_merge['Network_Entropy'] = profile_merge['Network_Entropy'].map(mapdict)

profile_merge.drop('Student_Count_Other_Ethnicity', axis=1, inplace=True)
profile_merge['Adjusted_Counts'] = profile_merge['Student_Count_Total'] - profile_merge['Student_Count_Ethnicity_Not_Available']
profile_merge.drop('Student_Count_Ethnicity_Not_Available', axis=1, inplace=True)

In [76]:
row_total = []
for i in range(len(profile_merge.index)):
    school_total = profile_merge.iloc[i, 92]
    x = profile_merge.iloc[i, 31:39]
    row_total.append(sum([(i/school_total)*np.log(1/(i/school_total)) for i in x if i!=0]))

profile_merge['School_Entropy'] = np.array(row_total)
profile_merge['District_Student_Count'] = profile_merge['Network_x']

In [77]:
count_1 = df.loc['Network 1', 'Totals-Ethnicity_NA']
count_2 = df.loc['Network 2', 'Totals-Ethnicity_NA']
count_3 = df.loc['Network 3', 'Totals-Ethnicity_NA']
count_4 = df.loc['Network 4', 'Totals-Ethnicity_NA']
count_5 = df.loc['Network 5', 'Totals-Ethnicity_NA']
count_6 = df.loc['Network 6', 'Totals-Ethnicity_NA']
count_7 = df.loc['Network 7', 'Totals-Ethnicity_NA']
count_8 = df.loc['Network 8', 'Totals-Ethnicity_NA']
count_9 = df.loc['Network 9', 'Totals-Ethnicity_NA']
count_10 = df.loc['Network 10', 'Totals-Ethnicity_NA']
count_11 = df.loc['Network 11', 'Totals-Ethnicity_NA']
count_12 = df.loc['Network 12', 'Totals-Ethnicity_NA']
count_13 = df.loc['Network 13', 'Totals-Ethnicity_NA']
count_ausl = df.loc['AUSL', 'Totals-Ethnicity_NA']
count_charter = df.loc['Charter', 'Totals-Ethnicity_NA']
count_contract = df.loc['Contract', 'Totals-Ethnicity_NA']
count_isp = df.loc['ISP', 'Totals-Ethnicity_NA']
count_options = df.loc['Options', 'Totals-Ethnicity_NA']

mapdict_count = {'Network 1': count_1, 'Network 2': count_2, 'Network 3': count_3,
          'Network 4': count_4, 'Network 5': count_5, 'Network 6': count_6,
          'Network 7': count_7, 'Network 8': count_8, 'Network 9': count_9, 
          'Network 10': count_10, 'Network 11': count_11, 'Network 12': count_12,
          'Network 13': count_13, 'AUSL': count_ausl, 'Charter': count_charter, 'Contract': count_contract, 
          'ISP': count_isp, 'Options': count_options}
profile_merge['District_Student_Count'] = profile_merge['District_Student_Count'].map(mapdict_count)

In [78]:
profile_merge['School_Theil'] = (profile_merge['Adjusted_Counts']*(profile_merge['Network_Entropy']-profile_merge['School_Entropy'])
                    /(profile_merge['Network_Entropy']*profile_merge['District_Student_Count']))

In [79]:
grouped_net = profile_merge.groupby('Network_x')
network_theil = pd.DataFrame(grouped_net['School_Theil'].sum())
theil_13 = network_theil.reset_index()
theil_13 = theil_13.drop([0, 1, 2, 3, 17])
theil_13['Network_x'] = theil_13['Network_x'].str.replace('Network ', '')

In [80]:
profile_merge

,Network_x,School_ID,Legacy_Unit_ID,Finance_ID,Short_Name,Long_Name,Primary_Category,Is_High_School,Is_Middle_School,Is_Elementary_School,Is_Pre_School,Summary,Administrator_Title,Administrator,Secondary_Contact_Title,Secondary_Contact,Phone,Fax,CPS_School_Profile,Website,Facebook,Twitter,Youtube,Pinterest,Attendance_Boundaries,Grades_Offered_All,Grades_Offered,Student_Count_Total,Student_Count_Low_Income,Student_Count_Special_Ed,Student_Count_English_Learners,Student_Count_Black,Student_Count_Hispanic,Student_Count_White,Student_Count_Asian,Student_Count_Native_American,Student_Count_Asian_Pacific_Islander,Student_Count_Multi,Student_Count_Hawaiian_Pacific_Islander,Statistics_Description,Demographic_Description,Dress_Code,PreK_School_Day,Kindergarten_School_Day,School_Hours,Freshman_Start_End_Time,After_School_Hours,Earliest_Drop_Off_Time,Classroom_Languages,Bilingual_Services,Refugee_Services,Title_1_Eligible,PreSchool_Inclusive,Preschool_Instructional,Significantly_Modified,Hard_Of_Hearing,Visual_Impairments,Transportation_Bus,Transportation_El,Transportation_Metra,Average_ACT_School,Mean_ACT,College_Enrollment_Rate_School,College_Enrollment_Rate_Mean,Graduation_Rate_School,Graduation_Rate_Mean,Overall_Rating,Rating_Status,Rating_Statement,Classification_Description,School_Year,Third_Contact_Title,Third_Contact_Name,Fourth_Contact_Title,Fourth_Contact_Name,Fifth_Contact_Title,Fifth_Contact_Name,Sixth_Contact_Title,Sixth_Contact_Name,Seventh_Contact_Title,Seventh_Contact_Name,Network_y,Is_GoCPS_Participant,Is_GoCPS_PreK,Is_GoCPS_Elementary,Is_GoCPS_High_School,Open_For_Enrollment_Date,Closed_For_Enrollment_Date,School_Latitude,School_Longitude,Location,Network_Entropy,Adjusted_Counts,School_Entropy,District_Student_Count,School_Theil
0,Network 1,610212,6290,32011,ALBANY PARK,Albany Park Multicultural Academy,MS,N,Y,N,N,Albany Park Multicultural Academy (APMA) offer...,Principal,Hiliana Araceli Leon,Assistant Principal,Marie A. Garza-Hammerlund,7.735345e+09,7.735345e+09,http://cps.edu/Schools/Pages/school.aspx?Schoo...,http://albanypark.cps.edu,NaN,https://twitter.com/APMABulldogs,NaN,NaN,Y,"7,8",7-8,281,185,47,73,14,224,14,24,3,0,2,0,There are 281 students enrolled at ALBANY PARK...,The largest demographic at ALBANY PARK is Hisp...,Y,NaN,NaN,08:00 AM-03:00 PM,NaN,3:15-4:15,7:45,NaN,Y,NaN,Y,NaN,NaN,NaN,NaN,NaN,NaN,Brown,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Level 1,GOOD STANDING,"This school received a Level 1 rating, which i...",Schools that have an attendance boundary. Gene...,School Year 2017-2018,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Network 1,Y,N,Y,N,09/01/2004 12:00:00 AM,NaN,41.971504,-87.710609,"4929 N SAWYER AVE\nChicago, Illinois 60625\n(4...",1.188612,281,0.773380,34353,0.002858
1,Network 1,610209,6270,25681,VOLTA,Alessandro Volta Elementary School,ES,N,Y,Y,Y,Volta offers a full spectrum of educational se...,Principal,Ms.Emily Mariano,Assistant Principal,A. Thepsiree,7.735345e+09,7.735345e+09,http://cps.edu/Schools/Pages/school.aspx?Schoo...,http://www.volta.cps.edu,NaN,NaN,NaN,NaN,Y,"PK,K,1,2,3,4,5,6,7,8","PK,K-8",825,494,79,543,16,570,44,190,0,0,3,1,There are 825 students enrolled at VOLTA. 59....,The largest demographic at VOLTA is Hispanic. ...,Y,Full Day,Full Day,8:00 am-3:00 pm,NaN,NaN,NaN,"Arabic, Spanish",Y,NaN,Y,NaN,NaN,NaN,NaN,NaN,"53, 81",Brown,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Level 1,GOOD STANDING,"This school received a Level 1 rating, which i...",Schools that have an attendance boundary. Gene...,School Year 2017-2018,Counselor,Katie Styzek,Dual Language Coordinator & EL Coach,Josue Contreras,English Language Learner Teacher (ELPT),Dipal Parekh,NaN,NaN,NaN,NaN,Network 1,Y,N,Y,N,09/01/2004 12:00:00 AM,NaN,41.971634,-87.724605,"4950 N AVERS AVE\nChicago, Illinois 60625\n(41...",1.188612,824,0.854815,34353,0.006736
2,Network 1,609836,2620,22541,CANTY,Arthur E Canty Elementary School,ES,N,Y,Y,Y,Canty School offers a variety of rigorous acad...,Principal,Sandra Kay Carlson,Assistant Principal,Miss Collette D. Laurencell

In [81]:
profile_2017 = pd.read_csv('Chicago_Public_Schools_-_School_Profile_Information_SY1617.csv')
header2 = ['Network', 'School_ID', 'School_Name', 'Total', 'White_Student_Count', 
          'White_Student_Pct', 'African_American_Count', 'African_America_Pct', 
         'AAPI_Count', 'AAPI_Pct', 'Native_American_Count', 'Native_American_Pct',
          'Hispanic_Student_Count', 'Hispanic_Student_Pct', 
          'Multi-Racial_Student_Count', 'Multi-Racial_Student_Pct', 
          'Asian_Student_Count', 'Asian_Student_Pct', 'Hawaiian_PI_Count', 
          'Hawaiian_PI_Pct', 'Not_Available_Count', 'Not_Available_Pct']

In [82]:
demo_data_2017 = pd.read_excel('Demographics_RacialEthnic_2017.xls', 'Schools', skiprows=2, names = header2)
demo_data_2018['School_ID'].astype(int, inplace=True)


0      610212
1      610209
2      609836
3      609729
4      610083
5      610564
6      610515
7      609901
8      610523
9      610137
10     609874
11     610163
12     609737
13     609857
14     610182
15     609972
16     609798
17     609766
18     609912
19     609796
20     610089
21     610111
22     609937
23     610159
24     610127
25     609792
26     609810
27     610354
28     609749
29     609744
        ...  
630    400176
631    610572
632    610573
633    400164
634    400137
635    610569
636    610570
637    610571
638    610557
639    610568
640    400173
641    400123
642    400133
643    400124
644    400125
645    400126
646    400127
647    400128
648    400150
649    400130
650    400131
651    400134
652    400135
653    400136
654    400129
655    400139
656    400141
657    400143
658    400144
659    400145
Name: School_ID, Length: 660, dtype: int64

In [83]:
demo_data_2017

,Network,School_ID,School_Name,Total,White_Student_Count,White_Student_Pct,African_American_Count,African_America_Pct,AAPI_Count,AAPI_Pct,Native_American_Count,Native_American_Pct,Hispanic_Student_Count,Hispanic_Student_Pct,Multi-Racial_Student_Count,Multi-Racial_Student_Pct,Asian_Student_Count,Asian_Student_Pct,Hawaiian_PI_Count,Hawaiian_PI_Pct,Not_Available_Count,Not_Available_Pct
0,Network 1,610212,Albany Park Multicultural Academy,277,18,6.5,12,4.3,0,0.0000,0,0.0000,216,78.0,4,1.44,27,9.747,0,0.0000,0,0.000
1,Network 1,610209,Alessandro Volta Elementary School,921,62,6.7,28,3.0,0,0.0000,1,0.1086,599,65.0,5,0.54,226,24.539,0,0.0000,0,0.000
2,Network 1,609836,Arthur E Canty Elementary School,849,442,52.1,8,0.9,0,0.0000,1,0.1178,340,40.0,25,2.94,31,3.651,1,0.1178,1,0.118
3,Network 1,609729,Carl Schurz High School,1812,106,5.8,200,11.0,0,0.0000,8,0.4415,1431,79.0,17,0.94,26,1.435,4,0.2208,20,1.104
4,Network 1,610083,Daniel C Beard Elementary School,173,76,43.9,7,4.0,0,0.0000,0,0.0000,73,42.2,6,3.47,11,6.358,0,0.0000,0,0.000
5,Network 1,610564,Disney II Magnet High School,730,159,21.8,144,19.7,0,0.0000,3,0.4110,378,51.8,15,2.05,27,3.699,3,0.4110,1,0.137
6,Network 1,610515,Disney II Magnet School,428,186,43.5,37,8.6,0,0.0000,2,0.4673,164,38.3,9,2.10,23,5.374,3,0.7009,4,0.935
7,Network 1,610523,Edison Park Elementary School,564,392,69.5,5,0.9,0,0.0000,1,0.1773,122,21.6,5,0.89,33,5.851,1,0.1773,5,0.887
8,Network 1,610137,Ernst Prussing Elementary School,705,331,47.0,15,2.1,0,0.0000,1,0.1418,284,40.3,8,1.13,49,6.950,6,0.8511,11,1.560
9,Network 1,609874,Everett McKinley Dirksen Elementary School,884,612,69.2,28,3.2,0,0.0000,2,0.2262,133,15.0,5,0.57,95,10.747,4,0.4525,5,0.566


In [104]:
profile_merge_2017 = pd.merge(demo_data_2017[['Network', 'School_ID']], profile_2017, how='inner', on='School_ID')

In [108]:
profile_merge_2017

,Network,School_ID,Legacy_Unit_ID,Finance_ID,Short_Name,Long_Name,School_Type,Primary_Category,Is_High_School,Is_Middle_School,Is_Elementary_School,Is_Pre_School,Summary,Administrator_Title,Administrator,Secondary_Contact_Title,Secondary_Contact,Address,City,State,Zip,Phone,Fax,CPS_School_Profile,Website,Facebook,Twitter,Youtube,Pinterest,Attendance_Boundaries,Grades_Offered_All,Grades_Offered,Student_Count_Total,Student_Count_Low_Income,Student_Count_Special_Ed,Student_Count_English_Learners,Student_Count_Black,Student_Count_Hispanic,Student_Count_White,Student_Count_Asian,Student_Count_Native_American,Student_Count_Other_Ethnicity,Student_Count_Asian_Pacific_Islander,Student_Count_Multi,Student_Count_Hawaiian_Pacific_Islander,Student_Count_Ethnicity_Not_Available,Statistics_Description,Demographic_Description,ADA_Accessible,Dress_Code,PreK_School_Day,Kindergarten_School_Day,School_Hours,Freshman_Start_End_Time,After_School_Hours,Earliest_Drop_Off_Time,Classroom_Languages,Bilingual_Services,Refugee_Services,Title_1_Eligible,PreSchool_Inclusive,Preschool_Instructional,Significantly_Modified,Hard_Of_Hearing,Visual_Impairments,Transportation_Bus,Transportation_El,Transportation_Metra,School_Latitude,School_Longitude,Average_ACT_School,Mean_ACT,College_Enrollment_Rate_School,College_Enrollment_Rate_Mean,Graduation_Rate_School,Graduation_Rate_Mean,Overall_Rating,Rating_Status,Rating_Statement,Classification_Description,School_Year,Third_Contact_Title,Third_Contact_Name,Fourth_Contact_Title,Fourth_Contact_Name,Fifth_Contact_Title,Fifth_Contact_Name,Sixth_Contact_Title,Sixth_Contact_Name,Seventh_Contact_Title,Seventh_Contact_Name,Location
0,Network 1,610212,6290,32011,ALBANY PARK,Albany Park Multicultural Academy,Neighborhood,MS,N,Y,N,N,Albany Park Multicultural Academy (APMA) offer...,Principal,Hiliana Araceli Leon,Assistant Principal,Marie A. Garza-Hammerlund,4929 N SAWYER AVE,Chicago,Illinois,60625,7735345108,7.735345e+09,http://cps.edu/Schools/Pages/school.aspx?Schoo...,http://albanypark.cps.edu,NaN,https://twitter.com/APMABulldogs,NaN,NaN,Y,"7,8",7-8,289,277,52,66,16,227,15,24,3,0,0,4,0,0,There are 289 students enrolled at ALBANY PARK...,The largest demographic at ALBANY PARK is Hisp...,Fully Accessible,Y,NaN,NaN,08:00 AM-03:00 PM,NaN,3:15-4:15,7:45,NaN,Y,NaN,Y,NaN,NaN,NaN,NaN,NaN,NaN,Brown,NaN,41.971504,-87.710609,NaN,18.4,NaN,57.9,NaN,73.5,Level 1+,Good Standing,"This school received a Level 1+ rating, which ...",Schools that have an attendance boundary. Gene...,School Year 2016-2017,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"4929 N SAWYER AVE\nChicago, Illinois 60625\n(4..."
1,Network 1,610209,6270,25681,VOLTA,Alessandro Volta Elementary School,Neighborhood,ES,N,Y,Y,Y,Volta offers a full spectrum of educational se...,Principal,Ms.Emily Mariano,Assistant Principal,A. Thepsiree,4950 N AVERS AVE,Chicago,Illinois,60625,7735345080,7.735345e+09,http://cps.edu/Schools/Pages/school.aspx?Schoo...,http://www.volta.cps.edu,NaN,NaN,NaN,NaN,Y,"PK,K,1,2,3,4,5,6,7,8","PK,K-8",891,748,88,518,27,577,59,221,0,0,0,5,1,1,There are 891 students enrolled at VOLTA. 84%...,The largest demographic at VOLTA is Hispanic. ...,No/unknown accessibility,Y,Half Day,Full Day,8:00 am-3:00 pm,NaN,NaN,NaN,"Arabic, Spanish",Y,NaN,Y,NaN,NaN,NaN,NaN,NaN,"53, 81",Brown,NaN,41.971634,-87.724605,NaN,18.4,NaN,57.9,NaN,73.5,Level 1,Good Standing,"This school received a Level 1 rating, which i...",Schools that have an attendance boundary. Gene...,School Year 2016-2017,Counselor,Katie Styzek,Dual Language/ELL Liason Coordinator,Guadalupe Sandoval,NaN,NaN,NaN,NaN,NaN,NaN,"4950 N AVERS AVE\nChicago, Illinois 60625\n(41..."
2,Network 1,609836,2620,22541,CANTY,Arthur E Canty Elementary School,Neighborhood,ES,N,Y,Y,Y,Canty School offers a variety of rigorous acad...,Principal,Lucja Mirowska-Kopec,Assistant Principal,Miss Collette D. Laurencell,3740 N PANAMA AVE,Chicago,Illinois,60634,7735341238,7.735341e+09,http://cps.edu/Schools/Pages/school.aspx?Schoo...,http://www.cantyscho

In [96]:
profile_2019 = pd.read_csv('Chicago_Public_Schools_-_School_Profile_Information_SY1819.csv')

In [97]:
header = ['Network', 'School_ID', 'School_Name', 'Total', 'White_Student_Count', 
          'White_Student_Pct', 'African_American_Count', 'African_America_Pct', 
         'AAPI_Count', 'AAPI_Pct', 'Native_American_Count', 'Native_American_Pct',
          'Hispanic_Student_Count', 'Hispanic_Student_Pct', 
          'Multi-Racial_Student_Count', 'Multi-Racial_Student_Pct', 
          'Asian_Student_Count', 'Asian_Student_Pct', 'Hawaiian_PI_Count', 
          'Hawaiian_PI_Pct', 'Not_Available_Count', 'Not_Available_Pct']
demo_data_2019 = pd.read_excel('Demographics_RacialEthnic_2019.xls', 'Schools', skiprows=3, names = header)
demo_data_2019['School_ID'].astype(int, inplace=True)
profile_merge_2019 = pd.merge(demo_data_2019[['Network', 'School_ID']], profile_2019, how='inner', on='School_ID')

In [109]:
charter_2018 = profile_merge[profile_merge['Network_x']=='Charter']
charter_2017 = profile_merge_2017[profile_merge_2017['Network']=='Charter']
charter_2019 = profile_merge_2019[profile_merge_2019['Network_x']=='Charter']

In [120]:
set(charter_2017['School_ID']).difference(set(charter_2019['School_ID']))

{400010}

In [114]:
charter_2017['']

,Network,School_ID,Legacy_Unit_ID,Finance_ID,Short_Name,Long_Name,School_Type,Primary_Category,Is_High_School,Is_Middle_School,Is_Elementary_School,Is_Pre_School,Summary,Administrator_Title,Administrator,Secondary_Contact_Title,Secondary_Contact,Address,City,State,Zip,Phone,Fax,CPS_School_Profile,Website,Facebook,Twitter,Youtube,Pinterest,Attendance_Boundaries,Grades_Offered_All,Grades_Offered,Student_Count_Total,Student_Count_Low_Income,Student_Count_Special_Ed,Student_Count_English_Learners,Student_Count_Black,Student_Count_Hispanic,Student_Count_White,Student_Count_Asian,Student_Count_Native_American,Student_Count_Other_Ethnicity,Student_Count_Asian_Pacific_Islander,Student_Count_Multi,Student_Count_Hawaiian_Pacific_Islander,Student_Count_Ethnicity_Not_Available,Statistics_Description,Demographic_Description,ADA_Accessible,Dress_Code,PreK_School_Day,Kindergarten_School_Day,School_Hours,Freshman_Start_End_Time,After_School_Hours,Earliest_Drop_Off_Time,Classroom_Languages,Bilingual_Services,Refugee_Services,Title_1_Eligible,PreSchool_Inclusive,Preschool_Instructional,Significantly_Modified,Hard_Of_Hearing,Visual_Impairments,Transportation_Bus,Transportation_El,Transportation_Metra,School_Latitude,School_Longitude,Average_ACT_School,Mean_ACT,College_Enrollment_Rate_School,College_Enrollment_Rate_Mean,Graduation_Rate_School,Graduation_Rate_Mean,Overall_Rating,Rating_Status,Rating_Statement,Classification_Description,School_Year,Third_Contact_Title,Third_Contact_Name,Fourth_Contact_Title,Fourth_Contact_Name,Fifth_Contact_Title,Fifth_Contact_Name,Sixth_Contact_Title,Sixth_Contact_Name,Seventh_Contact_Title,Seventh_Contact_Name,Location
517,Charter,400009,2025,63011,GLOBAL CITIZENSHIP,Academy for Global Citizenship Charter School,Charter,ES,N,Y,Y,N,AGC is a dual-language International Baccalaur...,Director,Dr. Jennifer Moore,Admissions Officer,Maribel Mares,4647 W 47TH ST,Chicago,Illinois,60632,7735821100,7.735821e+09,http://cps.edu/Schools/Pages/school.aspx?Schoo...,http://www.agcchicago.org,https://www.facebook.com/AGC.Chicago,https://twitter.com/globalchicago,NaN,NaN,N,"K,1,2,3,4,5,6,7,8",K-8,463,312,48,165,22,408,30,0,2,0,0,1,0,0,There are 463 students enrolled at GLOBAL CITI...,The largest demographic at GLOBAL CITIZENSHIP ...,Fully Accessible,Y,NaN,Full Day,8:00 AM-4:00 PM,NaN,4:00 PM -5:15 PM,07:45 AM,Spanish,Y,NaN,Y,NaN,NaN,NaN,NaN,NaN,"47, 54, 62",Orange,NaN,41.807579,-87.740097,NaN,18.4,NaN,57.9,NaN,73.5,Level 1,Not Applicable,"This school received a Level 1 rating, which i...","Schools that are open to all Chicago children,...",School Year 2016-2017,Principal,Jennifer Moore,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"4647 W 47TH ST\nChicago, Illinois 60632\n(41.8..."
518,Charter,400010,7950,66291,ACE TECH HS,ACE Technical Charter School,Charter,HS,Y,N,N,N,ACE Tech aims to introduce students to archite...,Director,Cecilia Baader,Chief Operations Officer,John Ayers,5410 S STATE ST,Chicago,Illinois,60609,7735488705,7.735489e+09,http://cps.edu/Schools/Pages/school.aspx?Schoo...,http://www.acetechnical.org,https://www.facebook.com/pages/ACE-Technical-C...,https://twitter.com/ACETECHNICAL,https://www.youtube.com/channel/UCZE3LFvDhbN7o...,NaN,N,"9,10,11,12",9-12,314,302,67,22,259,49,4,0,0,0,0,1,1,0,There are 314 students enrolled at ACE TECH HS...,The largest demographic at ACE TECH HS is Blac...,Generally accessible,Y,NaN,NaN,7:40 AM-3:22 PM,7:40 a.m. - 3:22 p.m.,NaN,NaN,Spanish,Y,N,Y,NaN,NaN,NaN,NaN,NaN,"15, 29, 55","Brown, Purple, Red",NaN,41.796122,-87.625849,15.2,18.4,56.3,57.9,68.8,73.5,Level 2,Not Applicable,"This school received a Level 2 rating, which i...","Schools that are open to all Chicago children,...",School Year 2016-2017,Interim Principal,Cecilia Baader,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"5410 S STATE ST\nChicago, Illinois 60609\n(41...."
519,Charter,400011,4730,66151,LOCKE A,Alain Locke Charter School,Charter,ES,N,Y,Y,N,With a goal of preparing its students for coll...,Director,Mr. Patrick Love,NaN,NaN,3141 W JACKSON BLVD

In [117]:
charter_2017.loc[518, 'Rating_Statement']

'This school received a Level 2 rating, which is a low rating at CPS. This rating is based on the CPS School Quality Rating Policy (SQRP).  To learn more, visit:  http://cps.edu/sqrp .'

In [121]:
charter_2017.columns

Index(['Network', 'School_ID', 'Legacy_Unit_ID', 'Finance_ID', 'Short_Name',
       'Long_Name', 'School_Type', 'Primary_Category', 'Is_High_School',
       'Is_Middle_School', 'Is_Elementary_School', 'Is_Pre_School', 'Summary',
       'Administrator_Title', 'Administrator', 'Secondary_Contact_Title',
       'Secondary_Contact', 'Address', 'City', 'State', 'Zip', 'Phone', 'Fax',
       'CPS_School_Profile', 'Website', 'Facebook', 'Twitter', 'Youtube',
       'Pinterest', 'Attendance_Boundaries', 'Grades_Offered_All',
       'Grades_Offered', 'Student_Count_Total', 'Student_Count_Low_Income',
       'Student_Count_Special_Ed', 'Student_Count_English_Learners',
       'Student_Count_Black', 'Student_Count_Hispanic', 'Student_Count_White',
       'Student_Count_Asian', 'Student_Count_Native_American',
       'Student_Count_Other_Ethnicity', 'Student_Count_Asian_Pacific_Islander',
       'Student_Count_Multi', 'Student_Count_Hawaiian_Pacific_Islander',
       'Student_Count_Ethnicity_Not_A

In [148]:
set(profile_merge['School_ID']).difference(set(profile_merge_2019['School_ID']))
# 3 School Openings : {610588: 10, 610589:8, 610590: 9}
#  6 School Closings: {400010:Charter, 609707:11, 609826: 7, 610012:6, 610212:6, 610506:11}

{610212, 610506}

In [146]:
profile_merge_2017[profile_merge_2017['School_ID']== 610012]

,Network,School_ID,Legacy_Unit_ID,Finance_ID,Short_Name,Long_Name,School_Type,Primary_Category,Is_High_School,Is_Middle_School,Is_Elementary_School,Is_Pre_School,Summary,Administrator_Title,Administrator,Secondary_Contact_Title,Secondary_Contact,Address,City,State,Zip,Phone,Fax,CPS_School_Profile,Website,Facebook,Twitter,Youtube,Pinterest,Attendance_Boundaries,Grades_Offered_All,Grades_Offered,Student_Count_Total,Student_Count_Low_Income,Student_Count_Special_Ed,Student_Count_English_Learners,Student_Count_Black,Student_Count_Hispanic,Student_Count_White,Student_Count_Asian,Student_Count_Native_American,Student_Count_Other_Ethnicity,Student_Count_Asian_Pacific_Islander,Student_Count_Multi,Student_Count_Hawaiian_Pacific_Islander,Student_Count_Ethnicity_Not_Available,Statistics_Description,Demographic_Description,ADA_Accessible,Dress_Code,PreK_School_Day,Kindergarten_School_Day,School_Hours,Freshman_Start_End_Time,After_School_Hours,Earliest_Drop_Off_Time,Classroom_Languages,Bilingual_Services,Refugee_Services,Title_1_Eligible,PreSchool_Inclusive,Preschool_Instructional,Significantly_Modified,Hard_Of_Hearing,Visual_Impairments,Transportation_Bus,Transportation_El,Transportation_Metra,School_Latitude,School_Longitude,Average_ACT_School,Mean_ACT,College_Enrollment_Rate_School,College_Enrollment_Rate_Mean,Graduation_Rate_School,Graduation_Rate_Mean,Overall_Rating,Rating_Status,Rating_Statement,Classification_Description,School_Year,Third_Contact_Title,Third_Contact_Name,Fourth_Contact_Title,Fourth_Contact_Name,Fifth_Contact_Title,Fifth_Contact_Name,Sixth_Contact_Title,Sixth_Contact_Name,Seventh_Contact_Title,Seventh_Contact_Name,Location
174,Network 6,610012,4200,23951,JENNER,Edward Jenner Elementary Academy of the Arts,Neighborhood,ES,N,Y,Y,Y,Jenner offers Performing Arts Programs along w...,Principal,Mrs.Kelly Anne Shelton,NaN,NaN,1119 N CLEVELAND AVE,Chicago,Illinois,60610,7735348440,7.735348e+09,http://cps.edu/Schools/Pages/school.aspx?Schoo...,http://jenner.cps.edu,https://www.facebook.com/Jenner-Academy-of-the...,http://twitter.com/JAAPrincipal,NaN,NaN,Y,"PK,K,1,2,3,4,5,6,7,8","PK,K-8",243,220,34,0,239,3,0,0,0,0,0,1,0,0,There are 243 students enrolled at JENNER. 90...,The largest demographic at JENNER is Black. T...,Fully Accessible,Y,Full and Half Day,Full Day,08:30 AM-03:30 PM,NaN,NaN,8:15am,NaN,NaN,NaN,Y,NaN,NaN,NaN,NaN,NaN,"11, 70","Blue, Red",NaN,41.902055,-87.641056,NaN,18.4,NaN,57.9,NaN,73.5,Level 2+,Intensive Support,"This school received a Level 2+ rating, which ...",Schools that have an attendance boundary. Gene...,School Year 2016-2017,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"1119 N CLEVELAND AVE\nChicago, Illinois 60610\..."
